<a href="https://colab.research.google.com/github/DaisyXinyiHe/diabetes_mellitus_predict/blob/pre_processing/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

## Execute this if using google colab and connecting it with google drive
## If not using google colab, navigate the script to your local drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Navigate data to its location
%cd /content/drive/MyDrive/Colab Notebooks/WiDS_2021

/content/drive/MyDrive/Colab Notebooks/WiDS_2021


In [3]:
# Load libraries
import numpy as np
import pandas as pd

# Explore Training Dataset

In [25]:
# load training data

training_data = pd.read_csv('TrainingWiDS2021.csv')

In [ ]:
training_data.head(10)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,...,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,...,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
5,6,182697,83,67.0,27.555611,0,Caucasian,M,190.5,Direct Admit,Accident & Emergency,95,admit,Med-Surg ICU,0.000694,0,100.0,NaN,301.0,403.01,0,0,NaN,13.0,0.71,NaN,4.0,6.0,0.0,5.0,156.0,113.0,44.2,0,130.0,NaN,NaN,NaN,NaN,35.0,...,NaN,NaN,NaN,NaN,1.1,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
6,7,195124,83,59.0,57.451002,0,Caucasian,F,165.1,Operating Room,Accident & Emergency,95,admit,Med-Surg ICU,0.000694,0,156.6,NaN,108.0,203.01,0,0,NaN,18.0,0.78,1.0,4.0,6.0,0.0,5.0,197.0,133.0,33.5,1,138.0,43.0,43.0,370.0,7.42,53.0,...,10.7,10.7,33.5,33.5,NaN,NaN,NaN,NaN,278.0,278.0,4.2,4.2,135.0,135.0,5.9,5.9,43.0,43.0,7.42,7.42,370.0,370.0,370.0,370.000000,43.0,43.0,7.42,7.42,370.0,370.0,370.0,370.0,0,0,0,0,0,0,0,1
7,8,179261,33,70.0,NaN,0,Caucasian,M,165.0,Emergency Department,Accident & Emergency,91,admit,Med-Surg ICU,0.0

In [ ]:
# Explore dataset
print(training_data.head(1))
print('training data size is:', training_data.shape)

   Unnamed: 0  encounter_id  ...  solid_tumor_with_metastasis  diabetes_mellitus
0           1        214826  ...                            0                  1

[1 rows x 181 columns]
training data size is: (130157, 181)


In [ ]:
# Check if feature variables have NA values
training_data = training_data.iloc[:,1:]
training_data.isnull().any()

encounter_id                   False
hospital_id                    False
age                             True
bmi                             True
elective_surgery               False
                               ...  
immunosuppression              False
leukemia                       False
lymphoma                       False
solid_tumor_with_metastasis    False
diabetes_mellitus              False
Length: 180, dtype: bool

In [ ]:
# check data types
print(training_data.dtypes)

encounter_id                     int64
hospital_id                      int64
age                            float64
bmi                            float64
elective_surgery                 int64
                                ...   
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
diabetes_mellitus                int64
Length: 180, dtype: object


In [ ]:
# check how many unique data types is in training_data
np.unique(training_data.dtypes)

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

#Check Duplication in training data**

In [27]:
training_data.duplicated().index

RangeIndex(start=0, stop=130157, step=1)

No duplicated data. Woohoo! But...is there duplicated weird data(e.g. a patient went to the hospital once, but data was stored twice in 2 different rows...but we wouldn't know until we have patient ID and time!)

In [30]:
# First, check if there are duplications on encounter ID
training_data['encounter_id'].duplicated().index

RangeIndex(start=0, stop=130157, step=1)

Ok. No duplicated patient encounter ID...probably no duplicated patient records. If there is, we couldn't know unless we have the patient ID and date information, which we do not have. 

# Clean Training Data

In [ ]:
# Check categorical data
for c in training_data.columns:
  if training_data[c].dtypes == np.dtype('O'):
      print(c)

ethnicity
gender
hospital_admit_source
icu_admit_source
icu_stay_type
icu_type


In [ ]:
# Check if independent variable have nan
print(training_data['diabetes_mellitus'].isnull().any)

<bound method Series.any of 0         False
1         False
2         False
3         False
4         False
          ...  
130152    False
130153    False
130154    False
130155    False
130156    False
Name: diabetes_mellitus, Length: 130157, dtype: bool>


In [ ]:
print(training_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'aids', 'cirrhosis',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'],
      dtype='object', length=180)


In [ ]:
# Assumming we want to keep all variables as predictors, 
# we need to create dummy variable all variables with NA values,  
# and change NA values to 0
# # Only do it to non-string variable for now
# Exclude ids and diabetes mellitus
for c in training_data.columns[3:]:
  if training_data[c].isnull().any() and training_data[c].dtypes != np.dtype('O'):


    newcol = c+'_0_1'
    newcol_value = np.array(training_data[c])


    nanindex = training_data[c].index[training_data[c].apply(np.isnan)]

    newcol_value[nanindex] = 0

    newcol_value[newcol_value!= 0]  = 1

    training_data[str(newcol)] = newcol_value


In [ ]:
print(training_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'd1_pao2fio2ratio_max_0_1', 'd1_pao2fio2ratio_min_0_1',
       'h1_arterial_pco2_max_0_1', 'h1_arterial_pco2_min_0_1',
       'h1_arterial_ph_max_0_1', 'h1_arterial_ph_min_0_1',
       'h1_arterial_po2_max_0_1', 'h1_arterial_po2_min_0_1',
       'h1_pao2fio2ratio_max_0_1', 'h1_pao2fio2ratio_min_0_1'],
      dtype='object', length=335)


In [ ]:
# # Check categorical data (e.g. ethnicity or dtype('O'))
for c in training_data.columns:
  if training_data[c].isnull().any() and training_data[c].dtypes == np.dtype('O'):
    print(c)
    print(training_data[c].unique())
    print(len(training_data[c].index[training_data[c].apply(pd.isnull)])/len(training_data))

ethnicity
['Caucasian' nan 'Hispanic' 'African American' 'Asian' 'Native American'
 'Other/Unknown']
0.012192966955292454
gender
['M' 'F' nan]
0.0005070799111841853
hospital_admit_source
['Floor' 'Emergency Department' 'Operating Room' nan 'Direct Admit'
 'Other Hospital' 'Other ICU' 'ICU to SDU' 'Recovery Room'
 'Chest Pain Center' 'Step-Down Unit (SDU)' 'Acute Care/Floor' 'PACU'
 'Observation' 'ICU' 'Other']
0.2550611953256452
icu_admit_source
['Floor' 'Accident & Emergency' 'Operating Room / Recovery'
 'Other Hospital' 'Other ICU' nan]
0.0018439269497606736


In [ ]:
# Taking out "hospital_admit_source" and "icu_admit_source" because they don't seem to be relevant 
# to the prediction of diabete mellitus, and they have a lot of missing data
training_data = training_data.drop(columns=['icu_admit_source', 'hospital_admit_source'])


In [ ]:
# Putting all "nan" in ethnicity to "other/unknown"
nanindex = training_data['ethnicity'].index[training_data['ethnicity'].apply(pd.isnull)]
training_data['ethnicity'].iloc[nanindex] = 'Other/Unknown'


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
# Delete the rows if they have nan values in "gender". This is only 66 rows
nanindex = training_data['gender'].index[training_data['gender'].apply(pd.isnull)]
training_data = training_data.drop(nanindex)
print(training_data.shape)


(130091, 333)


In [ ]:
# Check all current categorical data
for c in training_data.columns:
  if training_data[c].dtypes == np.dtype('O'):
      print(c)



ethnicity
gender
icu_stay_type
icu_type


In [ ]:
print(training_data['icu_stay_type'].unique())
print(training_data['icu_type'].unique())

['admit' 'readmit' 'transfer']
['CTICU' 'Med-Surg ICU' 'CCU-CTICU' 'Neuro ICU' 'MICU' 'SICU'
 'Cardiac ICU' 'CSICU']


In [ ]:
# Get dummy variable for categorical data
# dummy_v = pd.get_dummies(training_data[['ethnicity','gender','icu_stay_type','icu_type']],drop_first=True)
training_data = pd.get_dummies(data=training_data, columns=['ethnicity','gender','icu_stay_type','icu_type'])

In [ ]:
training_data.head(5)

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,...,h1_potassium_min_0_1,h1_sodium_max_0_1,h1_sodium_min_0_1,h1_wbc_max_0_1,h1_wbc_min_0_1,d1_arterial_pco2_max_0_1,d1_arterial_pco2_min_0_1,d1_arterial_ph_max_0_1,d1_arterial_ph_min_0_1,d1_arterial_po2_max_0_1,d1_arterial_po2_min_0_1,d1_pao2fio2ratio_max_0_1,d1_pao2fio2ratio_min_0_1,h1_arterial_pco2_max_0_1,h1_arterial_pco2_min_0_1,h1_arterial_ph_max_0_1,h1_arterial_ph_min_0_1,h1_arterial_po2_max_0_1,h1_arterial_po2_min_0_1,h1_pao2fio2ratio_max_0_1,h1_pao2fio2ratio_min_0_1,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU
0,214826,118,68.0,22.732803,0,180.3,92,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
1,246060,81,77.0,27.421875,0,160.0,90,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,276985,118,25.0,31.952749,0,172.7,93,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,262220,118,81.0,22.635548,1,165.1,92,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,1,8.0,62.0,30.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
4,201746,33,19.0,NaN,0,188.0,91,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
training_data = training_data.fillna(0)

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,...,h1_potassium_min_0_1,h1_sodium_max_0_1,h1_sodium_min_0_1,h1_wbc_max_0_1,h1_wbc_min_0_1,d1_arterial_pco2_max_0_1,d1_arterial_pco2_min_0_1,d1_arterial_ph_max_0_1,d1_arterial_ph_min_0_1,d1_arterial_po2_max_0_1,d1_arterial_po2_min_0_1,d1_pao2fio2ratio_max_0_1,d1_pao2fio2ratio_min_0_1,h1_arterial_pco2_max_0_1,h1_arterial_pco2_min_0_1,h1_arterial_ph_max_0_1,h1_arterial_ph_min_0_1,h1_arterial_po2_max_0_1,h1_arterial_po2_min_0_1,h1_pao2fio2ratio_max_0_1,h1_pao2fio2ratio_min_0_1,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU
0,214826,118,68.0,22.732803,0,180.3,92,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,0.0,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,0.0,0.0,0.0,0.000,36.0,134.0,39.3,0.0000,0,14.1,46.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
1,246060,81,77.0,27.421875,0,160.0,90,0.927778,0,70.2,0.0,108.0,203.01,0,0,0.0,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.450,33.0,145.0,35.1,0.0000,1,12.7,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,276985,118,25.0,31.952749,0,172.7,93,0.000694,0,95.3,0.0,122.0,703.03,0,0,0.0,0.0,0.00,0.0,3.0,6.0,0.0,5.0,0.0,102.0,0.0,0,68.0,0.0,0.0,0.0,0.000,37.0,0.0,36.7,0.0000,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,262220,118,81.0,22.635548,1,165.1,92,0.000694,0,61.7,0.0,203.0,1206.03,1,0,0.0,0.0,0.00,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.390,4.0,0.0,34.8,0.0000,1,8.0,62.0,30.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
4,201746,33,19.0,0.000000,0,188.0,91,0.073611,0,0.0,0.0,119.0,601.01,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0,103.0,0.0,0.0,0.0,0.000,16.0,0.0,36.7,0.0000,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130152,164758,7,50.0,29.287256,0,175.3,1109,0.023611,0,90.0,0.0,110.0,104.01,0,0,0.0,0.0,0.00,0.0,4.0,6.0,0.0,5.0,0.0,106.0,0.0,0,120.0,0.0,0.0,0.0,0.000,50.0,0.0,36.2,1751.1552,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
130153,197653,7,79.0,29.653433,0,162.6,1106,0.009722,0,78.4,3.5,105.0,207.01,0,0,0.7,32.0,1.07,0.0,4.0,6.0,0.0,4.0,139.0,112.0,39.0,0,125.0,0.0,0.0,0.0,0.000,42.0,140.0,36.4,5724.7776,0,9.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
130154,219719,7,73.0,32.265371,0,177.

# Save Cleaned Training Data

In [ ]:
# Save cleaned training dataset
training_data.to_csv('training_data_clean.csv',index=False)

In [ ]:
# Define independent and dependent variables
y = training_data['diabetes_mellitus']
training_data = training_data.drop(columns=['diabetes_mellitus'])
# encounter_id and hospital_id should not be invluded as predictors
training_data = training_data.drop(columns=['encounter_id', 'hospital_id'])
print(y.shape)
print(training_data.shape)

(130091,)
(130091, 345)


# Explore Unlabeled Data

In [4]:
# Load unlabeled data
unlabeled_data =  pd.read_csv('UnlabeledWiDS2021.csv')

In [5]:
unlabeled_data.head(10)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,1,144740,10141,72,NaN,0,Caucasian,F,152.4,Floor,Accident & Emergency,82,admit,Med-Surg ICU,0.015278,0,NaN,2.8,110.0,104.01,0,0,1.9,44.0,1.49,NaN,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,NaN,NaN,NaN,NaN,31.0,...,26.0,14.5,14.5,42.8,42.8,NaN,NaN,NaN,NaN,173.0,173.0,4.9,4.9,132.0,132.0,5.6,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,2,141990,10141,86,NaN,0,Caucasian,F,175.3,Emergency Department,Accident & Emergency,82,admit,Med-Surg ICU,0.000000,0,NaN,NaN,117.0,106.01,0,0,NaN,19.0,0.92,NaN,4.0,6.0,0.0,5.0,73.0,116.0,NaN,0,41.0,NaN,NaN,NaN,NaN,53.0,...,NaN,NaN,NaN,NaN,NaN,2.2,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
2,3,142038,10141,72,NaN,0,Caucasian,F,162.6,Floor,Floor,82,admit,Med-Surg ICU,0.003472,0,NaN,3.2,302.0,109.01,0,0,0.9,24.0,0.77,NaN,4.0,6.0,0.0,5.0,84.0,104.0,34.8,0,117.0,NaN,NaN,NaN,NaN,38.0,...,32.0,11.8,11.8,36.6,36.6,2.9,2.4,NaN,NaN,243.0,243.0,3.6,3.6,141.0,141.0,6.6,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
3,4,138628,10141,66,NaN,0,Caucasian,M,177.8,Floor,Floor,82,admit,Med-Surg ICU,0.884028,0,NaN,NaN,113.0,501.05,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,110.0,NaN,0,61.0,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
4,5,141682,10141,89,NaN,0,Caucasian,M,170.2,Direct Admit,Accident & Emergency,82,admit,Med-Surg ICU,0.013194,0,NaN,3.4,117.0,106.01,0,0,0.4,26.0,1.50,NaN,4.0,6.0,0.0,5.0,99.0,34.0,33.0,0,136.0,NaN,NaN,NaN,NaN,35.0,...,30.0,10.7,10.7,33.0,33.0,1.1,1.1,NaN,NaN,154.0,154.0,4.0,4.0,133.0,133.0,6.2,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
5,6,139096,10141,75,NaN,0,Caucasian,M,172.7,Emergency Department,Accident & Emergency,82,admit,Med-Surg ICU,0.002778,0,NaN,3.4,117.0,106.01,0,0,1.1,29.0,1.51,NaN,4.0,6.0,0.0,5.0,136.0,132.0,36.6,0,58.0,NaN,NaN,NaN,NaN,40.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
6,7,142994,10141,38,NaN,0,Caucasian,F,NaN,Floor,Floor,82,admit,Med-Surg ICU,15.888889,0,NaN,2.0,110.0,104.01,0,0,0.3,37.0,2.90,NaN,4.0,6.0,0.0,5.0,74.0,92.0,27.7,0,52.0,NaN,NaN,NaN,NaN,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
7,8,141954,10141,87,NaN,0,Caucasian,F,152.4,Floor,Floor,82,admit,Med-Surg ICU,0.056944,0,NaN,NaN,124.0,305.02,0,0,NaN,54.0,0.98,NaN,4.0,6.0,0.0,5.0,99.0,60.0,22.2,0,57.0,NaN,NaN,NaN,NaN,32.0,...,NaN,6.9,6.9,22.2,22.2,NaN,NaN,NaN,NaN,213.

In [6]:
# Explore dataset
print(unlabeled_data.head(1))
print('unlabeled data size is:', unlabeled_data.shape)

   Unnamed: 0  encounter_id  ...  lymphoma  solid_tumor_with_metastasis
0           1        144740  ...         0                            0

[1 rows x 180 columns]
unlabeled data size is: (10234, 180)


In [7]:
# Check if feature variables have NA values
unlabeled_data = unlabeled_data.iloc[:,1:]
unlabeled_data.isnull().any()

encounter_id                   False
hospital_id                    False
age                            False
bmi                             True
elective_surgery               False
                               ...  
hepatic_failure                False
immunosuppression              False
leukemia                       False
lymphoma                       False
solid_tumor_with_metastasis    False
Length: 179, dtype: bool

In [8]:
# check data types
print(unlabeled_data.dtypes)

encounter_id                     int64
hospital_id                      int64
age                              int64
bmi                            float64
elective_surgery                 int64
                                ...   
hepatic_failure                  int64
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
Length: 179, dtype: object


In [9]:
# check how many unique data types is in training_data
np.unique(unlabeled_data.dtypes)

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [10]:
# Check categorical data
for c in unlabeled_data.columns:
  if unlabeled_data[c].dtypes == np.dtype('O'):
      print(c)

ethnicity
gender
hospital_admit_source
icu_admit_source
icu_stay_type
icu_type


In [11]:
print(unlabeled_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'h1_arterial_po2_min', 'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min',
       'aids', 'cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia',
       'lymphoma', 'solid_tumor_with_metastasis'],
      dtype='object', length=179)


In [12]:
# Assumming we want to keep all variables as predictors, 
# we need to create dummy variable all variables with NA values,  
# and change NA values to 0
# # Only do it to non-string variable for now
# Exclude ids and diabetes mellitus
for c in unlabeled_data.columns[3:]:
  if unlabeled_data[c].isnull().any() and unlabeled_data[c].dtypes != np.dtype('O'):


    newcol = c+'_0_1'
    newcol_value = np.array(unlabeled_data[c])


    nanindex = unlabeled_data[c].index[unlabeled_data[c].apply(np.isnan)]

    newcol_value[nanindex] = 0

    newcol_value[newcol_value!= 0]  = 1

    unlabeled_data[str(newcol)] = newcol_value


In [13]:
print(unlabeled_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'd1_pao2fio2ratio_max_0_1', 'd1_pao2fio2ratio_min_0_1',
       'h1_arterial_pco2_max_0_1', 'h1_arterial_pco2_min_0_1',
       'h1_arterial_ph_max_0_1', 'h1_arterial_ph_min_0_1',
       'h1_arterial_po2_max_0_1', 'h1_arterial_po2_min_0_1',
       'h1_pao2fio2ratio_max_0_1', 'h1_pao2fio2ratio_min_0_1'],
      dtype='object', length=334)


In [14]:
# # Check categorical data (e.g. ethnicity or dtype('O'))
for c in unlabeled_data.columns:
  if unlabeled_data[c].isnull().any() and unlabeled_data[c].dtypes == np.dtype('O'):
    print(c)
    print(unlabeled_data[c].unique())
    print(len(unlabeled_data[c].index[unlabeled_data[c].apply(pd.isnull)])/len(unlabeled_data))

ethnicity
['Caucasian' nan 'Hispanic' 'Other/Unknown' 'African American' 'Asian'
 'Native American']
0.019933554817275746
gender
['F' 'M' nan]
0.0004885675200312683
hospital_admit_source
['Floor' 'Emergency Department' 'Direct Admit' nan 'Operating Room'
 'Other Hospital' 'Other ICU' 'ICU to SDU' 'Recovery Room'
 'Step-Down Unit (SDU)' 'Chest Pain Center']
0.26705100644909124
icu_admit_source
['Accident & Emergency' 'Floor' 'Operating Room / Recovery'
 'Other Hospital' nan 'Other ICU']
0.0024428376001563415


In [15]:
# Taking out "hospital_admit_source" and "icu_admit_source" since we took them out in training data
unlabeled_data = unlabeled_data.drop(columns=['icu_admit_source', 'hospital_admit_source'])


In [16]:
# Putting all "nan" in ethnicity to "other/unknown"
nanindex = unlabeled_data['ethnicity'].index[unlabeled_data['ethnicity'].apply(pd.isnull)]
unlabeled_data['ethnicity'].iloc[nanindex] = 'Other/Unknown'


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
# Delete the rows if they have nan values in "gender". This is only 66 rows
nanindex = unlabeled_data['gender'].index[unlabeled_data['gender'].apply(pd.isnull)]
unlabeled_data = unlabeled_data.drop(nanindex)
print(unlabeled_data.shape)


(10229, 332)


In [18]:
# Check all current categorical data
for c in unlabeled_data.columns:
  if unlabeled_data[c].dtypes == np.dtype('O'):
      print(c)



ethnicity
gender
icu_stay_type
icu_type


In [19]:
print(unlabeled_data['icu_stay_type'].unique())
print(unlabeled_data['icu_type'].unique())

['admit' 'readmit' 'transfer']
['Med-Surg ICU' 'SICU' 'CTICU' 'Neuro ICU' 'CCU-CTICU' 'MICU'
 'Cardiac ICU' 'CSICU']


In [20]:
# Get dummy variable for categorical data
unlabeled_data = pd.get_dummies(data=unlabeled_data, columns=['ethnicity','gender','icu_stay_type','icu_type'])

In [21]:
unlabeled_data.head(5)

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,...,h1_potassium_min_0_1,h1_sodium_max_0_1,h1_sodium_min_0_1,h1_wbc_max_0_1,h1_wbc_min_0_1,d1_arterial_pco2_max_0_1,d1_arterial_pco2_min_0_1,d1_arterial_ph_max_0_1,d1_arterial_ph_min_0_1,d1_arterial_po2_max_0_1,d1_arterial_po2_min_0_1,d1_pao2fio2ratio_max_0_1,d1_pao2fio2ratio_min_0_1,h1_arterial_pco2_max_0_1,h1_arterial_pco2_min_0_1,h1_arterial_ph_max_0_1,h1_arterial_ph_min_0_1,h1_arterial_po2_max_0_1,h1_arterial_po2_min_0_1,h1_pao2fio2ratio_max_0_1,h1_pao2fio2ratio_min_0_1,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU
0,144740,10141,72,NaN,0,152.4,82,0.015278,0,NaN,2.8,110.0,104.01,0,0,1.9,44.0,1.49,NaN,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,NaN,NaN,NaN,NaN,31.0,130.0,36.4,NaN,0,5.4,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
1,141990,10141,86,NaN,0,175.3,82,0.000000,0,NaN,NaN,117.0,106.01,0,0,NaN,19.0,0.92,NaN,4.0,6.0,0.0,5.0,73.0,116.0,NaN,0,41.0,NaN,NaN,NaN,NaN,53.0,142.0,36.3,NaN,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,142038,10141,72,NaN,0,162.6,82,0.003472,0,NaN,3.2,302.0,109.01,0,0,0.9,24.0,0.77,NaN,4.0,6.0,0.0,5.0,84.0,104.0,34.8,0,117.0,NaN,NaN,NaN,NaN,38.0,137.0,36.4,NaN,0,4.6,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,138628,10141,66,NaN,0,177.8,82,0.884028,0,NaN,NaN,113.0,501.05,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,110.0,NaN,0,61.0,NaN,NaN,NaN,NaN,56.0,NaN,36.6,NaN,1,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
4,141682,10141,89,NaN,0,170.2,82,0.013194,0,NaN,3.4,117.0,106.01,0,0,0.4,26.0,1.50,NaN,4.0,6.0,0.0,5.0,99.0,34.0,33.0,0,136.0,NaN,NaN,NaN,NaN,35.0,133.0,36.5,NaN,0,5.2,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0


In [22]:
unlabeled_data = unlabeled_data.fillna(0)

In [23]:
unlabeled_data.head(10)

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,...,h1_potassium_min_0_1,h1_sodium_max_0_1,h1_sodium_min_0_1,h1_wbc_max_0_1,h1_wbc_min_0_1,d1_arterial_pco2_max_0_1,d1_arterial_pco2_min_0_1,d1_arterial_ph_max_0_1,d1_arterial_ph_min_0_1,d1_arterial_po2_max_0_1,d1_arterial_po2_min_0_1,d1_pao2fio2ratio_max_0_1,d1_pao2fio2ratio_min_0_1,h1_arterial_pco2_max_0_1,h1_arterial_pco2_min_0_1,h1_arterial_ph_max_0_1,h1_arterial_ph_min_0_1,h1_arterial_po2_max_0_1,h1_arterial_po2_min_0_1,h1_pao2fio2ratio_max_0_1,h1_pao2fio2ratio_min_0_1,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU
0,144740,10141,72,0.0,0,152.4,82,0.015278,0,0.0,2.8,110.0,104.01,0,0,1.9,44.0,1.49,0.00,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,0.0,0.0,0.0,0.00,31.0,130.0,36.4,0.0,0,5.4,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
1,141990,10141,86,0.0,0,175.3,82,0.000000,0,0.0,0.0,117.0,106.01,0,0,0.0,19.0,0.92,0.00,4.0,6.0,0.0,5.0,73.0,116.0,0.0,0,41.0,0.0,0.0,0.0,0.00,53.0,142.0,36.3,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,142038,10141,72,0.0,0,162.6,82,0.003472,0,0.0,3.2,302.0,109.01,0,0,0.9,24.0,0.77,0.00,4.0,6.0,0.0,5.0,84.0,104.0,34.8,0,117.0,0.0,0.0,0.0,0.00,38.0,137.0,36.4,0.0,0,4.6,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,138628,10141,66,0.0,0,177.8,82,0.884028,0,0.0,0.0,113.0,501.05,0,0,0.0,0.0,0.00,0.00,4.0,6.0,0.0,4.0,0.0,110.0,0.0,0,61.0,0.0,0.0,0.0,0.00,56.0,0.0,36.6,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
4,141682,10141,89,0.0,0,170.2,82,0.013194,0,0.0,3.4,117.0,106.01,0,0,0.4,26.0,1.50,0.00,4.0,6.0,0.0,5.0,99.0,34.0,33.0,0,136.0,0.0,0.0,0.0,0.00,35.0,133.0,36.5,0.0,0,5.2,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
5,139096,10141,75,0.0,0,172.7,82,0.002778,0,0.0,3.4,117.0,106.01,0,0,1.1,29.0,1.51,0.00,4.0,6.0,0.0,5.0,136.0,132.0,36.6,0,58.0,0.0,0.0,0.0,0.00,40.0,139.0,36.7,0.0,0,8.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
6,142994,10141,38,0.0,0,0.0,82,15.888889,0,0.0,2.0,110.0,104.01,0,0,0.3,37.0,2.90,0.00,4.0,6.0,0.0,5.0,74.0,92.0,27.7,0,52.0,0.0,0.0,0.0,0.00,35.0,137.0,36.0,0.0,0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
7,141954,10141,87,0.0,0,152.4,82,0.056944,0,0.0,0.0,124.0,305.02,0,0,0.0,54.0,0.98,0.00,4.0,6.0,0.0,5.0,99.0,60.0,22.2,0,57.0,0.0,0.0,0.0,0.00,32.0,142.0,36.7,0.0,0,7.4,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
8,135344,10141,85,0.0,0,172.7,82,0.225000,0,0.0,1.8,113.0,501.05,0,0,0.6,34.0,1.44,0.00,3.0,6.0,0.0,5.0,264.0,100.0,37.2,0,44.0,0.0,0.0,0.0,0.00,53.0,1

# Save Cleaned Unlabeled Data

In [31]:
# Save cleaned training dataset
unlabeled_data.to_csv('unlabeled_data_clean.csv',index=False)